# CHO - Human ortholog genes dataset
This notebook describes the steps taken to obtain a dataset containing all the ortholog genes from Human and Hamster with its corresponding NIH IDs, Ensembl IDs (CHOK1GS assembly is used for CHO) and Gene Name (Symbol).

### Description
A dataset containing Ensemb IDs of ortholog genes between Human and Chinese Hamster was obtained from BioMart. Two  datasets containing a complete gene lists for the entire Human and Chinese Hamster organisms were obtained from NIH Datasets https://www.ncbi.nlm.nih.gov/datasets/. Furthermore, two datasets containing Ensembl and Entrez IDs from both, Human and CHO, were obtained from DAVID https://david.ncifcrf.gov/tools.jsp. The final "orthologs" dataset generated contains all the ortholog genes from Human and Hamster with its corresponding NIH IDs. Finally, one dataset containing CHO and Human gene orthologs from Paranoid database https://inparanoid.sbc.su.se was used to fill empty genes.

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
#Generate orthologs df obtained from the BioMart database
orthologs = pd.read_csv('../Data/GPR_Curation/mart_export-3.txt')
#Eliminate null values in the dataset, that represent genes with no reported orthologs
orthologs = orthologs.dropna()
#Drop useless columns 
orthologs = orthologs.drop(['Gene stable ID version'], axis=1)
# Change columns names
orthologs = orthologs.rename(columns={'Gene stable ID': 'CHO Ensembl ID','Gene name': 'CHO Gene Name','Human gene stable ID':'Human Ensembl ID','Human gene name':'Human Gene Name'})

orthologs

CHO Ensembl ID CHO Gene Name Human Ensembl ID Human Gene Name
5      ENSCGRG00001000006           ND1  ENSG00000198888          MT-ND1
9      ENSCGRG00001000010           ND2  ENSG00000198763          MT-ND2
15     ENSCGRG00001000016          COX1  ENSG00000198804          MT-CO1
18     ENSCGRG00001000019          COX2  ENSG00000198712          MT-CO2
20     ENSCGRG00001000021          ATP8  ENSG00000228253         MT-ATP8
...                   ...           ...              ...             ...
27530  ENSCGRG00001025067         Ero1b  ENSG00000086619           ERO1B
27531  ENSCGRG00001025068        Pcp4l1  ENSG00000248485          PCP4L1
27532  ENSCGRG00001025069       Slc30a1  ENSG00000170385         SLC30A1
27534  ENSCGRG00001025071        Zfp605  ENSG00000196458          ZNF605
27535  ENSCGRG00001025072        Slc3a2  ENSG00000168003          SLC3A2

[18157 rows x 4 columns]

Dictionaries from the NCBI datasets are used to map Entrez Gene IDs from the Gene Names

In [3]:
#Dict from dataset of Human Entrez Gene IDs from NCBI database
human_genes = pd.read_csv('../Data/GPR_Curation/human_ncbi_genes.txt', sep='\t')
human_genes = human_genes.rename(columns={'Symbol': 'Human Gene Name','NCBI GeneID': 'Human GeneID'})
human_dict = human_genes.set_index('Human Gene Name')['Human GeneID'].to_dict()
human_dict

{'TP53': 7157,
 'EGFR': 1956,
 'TNF': 7124,
 'APOE': 348,
 'VEGFA': 7422,
 'IL6': 3569,
 'TGFB1': 7040,
 'MTHFR': 4524,
 'ERBB2': 2064,
 'HIF1A': 3091,
 'ESR1': 2099,
 'IL10': 3586,
 'APP': 351,
 'ACE': 1636,
 'BRCA1': 672,
 'STAT3': 6774,
 'MMP9': 4318,
 'CRP': 1401,
 'KRAS': 3845,
 'BDNF': 627,
 'BRAF': 673,
 'ADIPOQ': 9370,
 'AR': 367,
 'AKT1': 207,
 'ABCB1': 5243,
 'HLA-DRB1': 3123,
 'NFKB1': 4790,
 'VDR': 7421,
 'IL1B': 3553,
 'CDKN2A': 1029,
 'PTEN': 5728,
 'LOC110806262': 110806262,
 'CTNNB1': 1499,
 'TLR4': 7099,
 'PTGS2': 5743,
 'NOS3': 4846,
 'CFTR': 1080,
 'COMT': 1312,
 'SNCA': 6622,
 'CXCL8': 3576,
 'CD274': 29126,
 'TERT': 7015,
 'PPARG': 5468,
 'SLC6A4': 6532,
 'MYC': 4609,
 'MAPT': 4137,
 'HLA-B': 3106,
 'MTOR': 2475,
 'CDH1': 999,
 'IGF1': 3479,
 'BCL2': 596,
 'LEP': 3952,
 'CXCR4': 7852,
 'GSTM1': 2944,
 'IFNG': 3458,
 'BRCA2': 675,
 'MDM2': 4193,
 'JAK2': 3717,
 'MMP2': 4313,
 'CD4': 920,
 'NFE2L2': 4780,
 'MAPK1': 5594,
 'CCND1': 595,
 'IL17A': 3605,
 'SERPINE1': 50

In [4]:
#Dict from dataset of CHO Entrez Gene IDs from NCBI database
cho_genes = pd.read_csv('../Data/GPR_Curation/cho_ncbi_genes.txt', sep='\t')
cho_genes = cho_genes.rename(columns={'Symbol': 'CHO Gene Name','NCBI GeneID': 'CHO GeneID'})
cho_dict = cho_genes.set_index('CHO Gene Name')['CHO GeneID'].to_dict()
cho_dict

{'Scap': 100689048,
 'Nsf': 100770898,
 'Abcb1': 100682536,
 'Dhfr': 100689028,
 'Ercc2': 100689272,
 'Tubb': 100689091,
 'Pea15': 100736554,
 'Sar1b': 100758826,
 'Pex5': 100689015,
 'Rad51c': 100689078,
 'Hspa5': 100689305,
 'Hmgcr': 100756363,
 'Tp53': 100682525,
 'Mgat1': 100682529,
 'Rpsa': 100689045,
 'Eef2': 100689051,
 'Slc20a2': 100689023,
 'Hs6st1': 100689173,
 'Ubb': 100689268,
 'Cbr3': 100689021,
 'Kif2c': 100689309,
 'Mt1': 100689478,
 'Mir7a': 102466467,
 'Mt2': 100689479,
 'Polr2a': 100689016,
 'Srebf2': 100689017,
 'Dpagt1': 100689054,
 'Casp3': 100689061,
 'Ldha': 100689064,
 'Cxcl3': 100689100,
 'Eef1a1': 100689276,
 'Rps14': 100689292,
 'Akr1b8': 100689318,
 'Xrcc1': 100689414,
 'Npc1': 100689424,
 'Ptdss2': 100689448,
 'Parp1': 100689463,
 'Smu1': 100736555,
 'Prnp': 100762792,
 'Kars1': 100766627,
 'Mpdu1': 100689049,
 'Mbtps1': 100689417,
 'Bdnf': 100768664,
 'Ap3m1': 100689277,
 'Adam17': 100684971,
 'Pisd': 100689093,
 'Ugp2': 100689325,
 'Ldlr': 100689399,
 'Hs

Dictionaries from the DAVID datasets are used to map Entrez Gene IDs from the Ensembl IDs

In [5]:
#Dict from dataset of Human Entrez Gene IDs from DAVID
human_david = pd.read_csv('../Data/GPR_Curation/david_output_human.txt', sep='\t')
human_dict2 = human_david.set_index('From')['To'].to_dict()
human_dict2

{'ENSG00000215203': 389207,
 'ENSG00000179029': 84314,
 'ENSG00000179023': 127707,
 'ENSG00000180011': 284273,
 'ENSG00000131089': 23229,
 'ENSG00000170689': 3219,
 'ENSG00000013810': 10460,
 'ENSG00000205884': 613210,
 'ENSG00000131080': 60401,
 'ENSG00000168374': 378,
 'ENSG00000179021': 285237,
 'ENSG00000156381': 122416,
 'ENSG00000182685': 283870,
 'ENSG00000182687': 8811,
 'ENSG00000143079': 55917,
 'ENSG00000180008': 122809,
 'ENSG00000025800': 23633,
 'ENSG00000143061': 3321,
 'ENSG00000156374': 84108,
 'ENSG00000131096': 5697,
 'ENSG00000181350': 388341,
 'ENSG00000131095': 2670,
 'ENSG00000131097': 51751,
 'ENSG00000168389': 84879,
 'ENSG00000131094': 10882,
 'ENSG00000182670': 7267,
 'ENSG00000168386': 11259,
 'ENSG00000144395': 284992,
 'ENSG00000143067': 90874,
 'ENSG00000182676': 116729,
 'ENSG00000180035': 197407,
 'ENSG00000155034': 80028,
 'ENSG00000179044': 283849,
 'ENSG00000118094': 11181,
 'ENSG00000179046': 205860,
 'ENSG00000157693': 203197,
 'ENSG00000168394': 6

In [6]:
#Dict from dataset of CHO Entrez Gene IDs from DAVID
cho_david = pd.read_csv('../Data/GPR_Curation/david_output_cho.txt', sep='\t')
cho_dict2 = cho_david.set_index('From')['To'].to_dict()
cho_dict2

{'ENSCGRG00001018393': 100767767,
 'ENSCGRG00001017062': 100770447,
 'ENSCGRG00001018390': 100759155,
 'ENSCGRG00001017061': 100751868,
 'ENSCGRG00001018391': 100753400,
 'ENSCGRG00001017060': 100756731,
 'ENSCGRG00001017067': 100764339,
 'ENSCGRG00001017066': 100756003,
 'ENSCGRG00001018399': 100769638,
 'ENSCGRG00001005074': 100754819,
 'ENSCGRG00001005078': 113837268,
 'ENSCGRG00001005076': 100769225,
 'ENSCGRG00001017073': 100759646,
 'ENSCGRG00001017076': 100756306,
 'ENSCGRG00001017070': 100754902,
 'ENSCGRG00001017071': 100760100,
 'ENSCGRG00001017078': 100756850,
 'ENSCGRG00001017079': 100762299,
 'ENSCGRG00001005061': 100751441,
 'ENSCGRG00001014803': 100770376,
 'ENSCGRG00001005062': 103161413,
 'ENSCGRG00001014809': 100763237,
 'ENSCGRG00001014805': 100755256,
 'ENSCGRG00001006396': 100763695,
 'ENSCGRG00001005065': 100752292,
 'ENSCGRG00001014807': 100771751,
 'ENSCGRG00001005066': 100757565,
 'ENSCGRG00001017085': 100756377,
 'ENSCGRG00001017087': 100765826,
 'ENSCGRG00001

In [7]:
#Dict from the paranoid dataset obtained by Hooman in 2016
# The dictionary generated has Human Gene IDs as keys and CHO Gene IDs "orthologs" as values
cho_paranoid_df = pd.read_csv('../Data/GPR_Curation/human.cho.id.final.with.mito.with.inparanoid', sep=';', names=['Human IDs', 'CHO IDs'])
cho_paranoid_df = cho_paranoid_df.astype(str)
cho_paranoid_dict = cho_paranoid_df.set_index('Human IDs')['CHO IDs'].to_dict()
cho_paranoid_dict

{'10000': '100752454',
 '1000': '100689204',
 '10001': '100762349',
 '10002': '100752298',
 '10003': '100769032',
 '10004': '100768106',
 '10005': '100761464',
 '10006': '100772463',
 '10007': '100689210',
 '10008': '100755499',
 '10009': '100763419',
 '100': '100756254',
 '10010': '100764984',
 '100101267': '100762026',
 '100101467': '100766295',
 '1001': '100758473',
 '10011': '100753253',
 '100113407': '100763981',
 '100125288': '100761549',
 '100127206': '100772872',
 '100127889': '100762880',
 '100127983': '100771996',
 '100128569': '100762642',
 '100128731': '100762772',
 '100128927': '100752108',
 '100129094': '100762433',
 '100129271': '100756558',
 '100129361': '103159661',
 '100129480': '100768732',
 '100129520': '100754139',
 '100129583': '100760554',
 '100129669': '100761208',
 '100129792': '100763254',
 '100129924': '100773888',
 '100130097': '103161207',
 '100130274': '100763932',
 '100130301': '100763417',
 '100130311': '100770028',
 '100130348': '100767526',
 '100130361

In [8]:
# Map dictionary into the orthologs dataset

# CHO Genes
orthologs['CHO GeneID_David'] = orthologs['CHO Ensembl ID'].map(cho_dict2)
orthologs['CHO GeneID_David'] = orthologs['CHO GeneID_David'].fillna('').astype(str).str.replace(".0","",regex=False)
orthologs['CHO GeneID'] = orthologs['CHO Gene Name'].map(cho_dict)
orthologs['CHO GeneID'] = orthologs['CHO GeneID'].fillna('').astype(str).str.replace(".0","",regex=False)

# Human Genes
orthologs['Human GeneID'] = orthologs['Human Gene Name'].map(human_dict)
orthologs['Human GeneID'] = orthologs['Human GeneID'].fillna('').astype(str).str.replace(".0","",regex=False)
orthologs['Human GeneID_David'] = orthologs['Human Ensembl ID'].map(cho_dict2)
orthologs['Human GeneID_David'] = orthologs['Human GeneID_David'].fillna('').astype(str).str.replace(".0","",regex=False)



#Replace empty values with the values obtained from David
for i,row in orthologs.iterrows():
    if row['CHO GeneID'] == '':
        row['CHO GeneID'] = row['CHO GeneID_David']
    if row['Human GeneID'] == '':
        row['Human GeneID'] = row['Human GeneID_David']
    
#Replace the rest of the empty values with the those from the paranoid dataset
for i,row in orthologs.iterrows():
    if row['CHO GeneID'] == '':
        try:
            row['CHO GeneID'] = cho_paranoid_dict[row['Human GeneID']]
        except:
            continue
    if row['Human GeneID'] == '':
        try:
            row['Human GeneID'] = [k for k, v in cho_paranoid_dict.items() if v == row['CHO GeneID']]
        except:
            continue

orthologs.drop(['CHO GeneID_David'], axis=1)        

orthologs['Human GeneID'] = orthologs['Human Gene Name'].map(human_dict)
orthologs['Human GeneID'] = orthologs['Human GeneID'].fillna('').astype(str).str.replace(".0","",regex=False)

# Reorganize columns
orthologs = orthologs[['CHO Gene Name','CHO GeneID', 'CHO Ensembl ID', 'Human Gene Name', 'Human GeneID','Human Ensembl ID']]
orthologs = orthologs.reset_index(drop = True)

orthologs.to_excel('../Data/GPR_Curation/orthologs.xlsx')
orthologs

CHO Gene Name CHO GeneID      CHO Ensembl ID Human Gene Name  \
0               ND1    3979183  ENSCGRG00001000006          MT-ND1   
1               ND2    3979184  ENSCGRG00001000010          MT-ND2   
2              COX1    3979185  ENSCGRG00001000016          MT-CO1   
3              COX2    3979186  ENSCGRG00001000019          MT-CO2   
4              ATP8    3979187  ENSCGRG00001000021         MT-ATP8   
...             ...        ...                 ...             ...   
18152         Ero1b  100764579  ENSCGRG00001025067           ERO1B   
18153        Pcp4l1  100770275  ENSCGRG00001025068          PCP4L1   
18154       Slc30a1  100751596  ENSCGRG00001025069         SLC30A1   
18155        Zfp605  100767561  ENSCGRG00001025071          ZNF605   
18156        Slc3a2  100689315  ENSCGRG00001025072          SLC3A2   

      Human GeneID Human Ensembl ID  
0             4535  ENSG00000198888  
1             4536  ENSG00000198763  
2             4512  ENSG00000198804  
3             4513  ENSG00000198712  
4             4509  ENSG00000228253  
...            ...              ...  
18152        56605  ENSG00000086619  
18153       654790  ENSG00000248485  
18154         7779  ENSG00000170385  
18155    100289635  ENSG00000196458  
18156         6520  ENSG00000168003  

[18157 rows x 6 columns]

#### The ortholog dataset generated here will then be used to map all the CHO gene IDs in Recon3D in the Recon3D notebook, map GO terms from CHO genes and gene descriptions from Human GeneIDs.